In [1]:
from datetime import date, timedelta
#from urllib.request import urlopen
import requests
import os
from bs4 import BeautifulSoup as soup
import time
import lxml
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import pdb

In [2]:
def initialize_browser(browser,start_date):
    #from selenium import webdriver
    ChromeDriver = 'C:\\Henry\\PythonProjects\\ChromeDriver\\'
    
    
    chromeOptions = webdriver.ChromeOptions()
    prefs={"profile.managed_default_content_settings.images": 2, 'disk-cache-size': 4096 }
    chromeOptions.add_experimental_option("prefs", prefs)
    chromeOptions.add_extension('C:\\Henry\\PythonProjects\\ChromeDriver\\Ublock\\ublock.zip')
    browser = webdriver.Chrome(chrome_options=chromeOptions)
    #browser.get()
    #us this url to scrape weather data for Davis, Ca
    url = "https://www.wunderground.com/history/daily/us/ca/davis/KSMF"

    browser.get(url)
    print('sleep')
    time.sleep(5)
    print('wake')
     # setup csv file to store the data
    
    output_file_name = 'C:\\Henry\\PythonProjects\\Weather\\Davis_Ca\\davis_weather.csv'
    print(output_file_name)
    fieldnames = ['date','high_temp', 'low_temp','average_temp']

    if os.path.exists(output_file_name):
        f_handle = open(output_file_name,  "a", newline='')
        csv_handle = csv.DictWriter(f_handle, fieldnames=fieldnames)
  
    else:
        f_handle = open(output_file_name,  "w", newline='')
        csv_handle = csv.DictWriter(f_handle, fieldnames=fieldnames)
        csv_handle.writeheader()
    
    error_flag = advance_date(start_date,browser,f_handle)
    
    
    return browser, f_handle,csv_handle

In [3]:
def scrape_station(start_date,browser,f_handle,csv_handle):
    '''
    This function scrapes the weather data web pages from wunderground.com
    from a predefined city (defined in initialize_browser())
    '''
    #make sure browser is on the start page
    url = "https://www.wunderground.com/history/daily/us/ca/davis/KSMF"
    

    # Scrape between start_date and end_date, hard coded
    
    end_date = date(year=2019, month=9, day=1)
    
    
    #setup dictionary to handle temperature data
    weather_info = {'date': 'unknown',
             'high_temp': 'unknown',
            'low_temp' : 'unknown',
            "average_temp": 'unknown'}
    
    
    #hard coded location where the data should be stored
    data_directory = 'C:\\Henry\\PythonProjects\\Weather\\'
    
    
   
    
    
    
    #determin if there is a folder for the current city. Currently hard coded to be Davis_Ca
    if not os.path.isdir('C:\\Henry\\PythonProjects\\Weather\\Davis_Ca'):
        os.mkdir('C:\\Henry\\PythonProjects\\Weather\\Davis_Ca')
    else:
        print('directory alreay exists')
    


    
    #loop to scrape data over multiple days
    current_date = start_date
    while current_date <= end_date:
        
            
            

        
        print(current_date)
  

        #sleep for 5 secpnds to be polite and wait for the dynamic page to load
        
        start_time = time.time()
        html = browser.page_source
        elapsed_time = time.time() - start_time
        print('page loading time = ' + str(elapsed_time) + ' seconds')
        print('sleep')
        time.sleep(5)
        print('awake')
        
        #scrape temprature data
        error_flag = 1
        attempt =0
        weather_info['date'] = current_date
        while error_flag ==1:
            attempt +=1
            weather_soup = soup(html, "lxml")
            weather_info,error_flag = get_daily_temp(weather_soup,weather_info)
            if error_flag ==1:
                print('data not present, wait 5 seconds and try again. Attempt' + str(attempt))
                break
            
        
        #write data to csv file, previously opened
        print(weather_info)
        csv_handle.writerow(weather_info)
        #time.sleep(3)
        
        
        #advance date one day
        current_date += timedelta(days=1)
        error_flag = advance_date(current_date,browser,f_handle)
        if error_flag > 0:
                print('error flag = ' + str(error_flag) + 'terminate scrape')
                break
    f_handle.close()
        
        
     

In [4]:
def get_daily_temp(weather_soup,weather_info):
    try:
        weather_data = weather_soup.findAll('div',{'class': 'summary-table'})
        weather_data[0].find_all('tr')[1].findAll('td')[0].text
        weather_info['high_temp'] = weather_data[0].find_all('tr')[1].findAll('td')[0].text #actual high temp
        weather_info['low_temp'] =weather_data[0].find_all('tr')[2].findAll('td')[0].text #actual low temp
        weather_info['average_temp'] =weather_data[0].find_all('tr')[3].findAll('td')[0].text #actual average temp
        error_flag =0
    except:
        error_flag =1
    return weather_info, error_flag

In [5]:
def advance_date(current_date,browser,f_handle):
    '''
    #this function advances the date on using the graphical interface on
    #https://www.wunderground.com/
    
    #required inputs: current date  = a datetime.date object, e.g., current_date  = date(year=2010, month=4, day=4)
                      browser a webdriver from selenium, e.g., browser = webdriver.Chrome()
    '''
    try:
        print(current_date)
        #find the select elements to enter data
        #get a list of all the select objects on the website 
        list_of_select = browser.find_elements(By.XPATH, '//select')
        # the first select element is the month
        list_of_select[0].find_elements_by_tag_name('option')[current_date.month-1].click()
        print('month selected')
        
        # the second select element is the day
        list_of_select[1].find_elements_by_tag_name('option')[current_date.day-1].click()
        print('day selected')
        
        #the third select element is the year
        list_of_select[2].find_elements_by_tag_name('option')[2019-current_date.year].click()
        print('year selected')

        #click button to view next page
        list_of_buttons = browser.find_elements(By.XPATH, "//input[@type = 'submit']")[0].click()
        error_flag=0
    except:
        print('error in advancing day,closing file')
        #pdb.set_trace()
        f_handle.close()
        error_flag =1
    
    return error_flag

In [28]:
start_date = date(year=2019, month=6, day=22)
browser = []
browser,f_handle,csv_handle = initialize_browser(browser,start_date)


C:\Users\hjali\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


sleep
wake
C:\Henry\PythonProjects\Weather\Davis_Ca\davis_weather.csv
2019-06-22
month selected
day selected
year selected


In [29]:
start_date = date(year=2019, month=6, day=22)
scrape_station(start_date,browser,f_handle,csv_handle)

directory alreay exists
2019-06-22
page loading time = 0.04094696044921875 seconds
sleep
awake
{'date': datetime.date(2019, 6, 22), 'high_temp': '92', 'low_temp': '56', 'average_temp': '74'}
2019-06-23
month selected
day selected
year selected
2019-06-23
page loading time = 0.056845664978027344 seconds
sleep
awake
{'date': datetime.date(2019, 6, 23), 'high_temp': '96', 'low_temp': '62', 'average_temp': '79'}
2019-06-24
month selected
day selected
year selected
2019-06-24
page loading time = 0.10671401023864746 seconds
sleep
awake
{'date': datetime.date(2019, 6, 24), 'high_temp': '92', 'low_temp': '62', 'average_temp': '77'}
2019-06-25
month selected
day selected
year selected
2019-06-25
page loading time = 0.04288458824157715 seconds
sleep
awake
{'date': datetime.date(2019, 6, 25), 'high_temp': '0', 'low_temp': '0', 'average_temp': '0'}
2019-06-26
month selected
day selected
year selected
2019-06-26
page loading time = 0.1107029914855957 seconds
sleep
awake
{'date': datetime.date(2019,

day selected
year selected
2019-07-29
page loading time = 0.13065028190612793 seconds
sleep
awake
{'date': datetime.date(2019, 7, 29), 'high_temp': '91', 'low_temp': '61', 'average_temp': '76'}
2019-07-30
month selected
day selected
year selected
2019-07-30
page loading time = 0.16655349731445312 seconds
sleep
awake
{'date': datetime.date(2019, 7, 30), 'high_temp': '88', 'low_temp': '57', 'average_temp': '72'}
2019-07-31
month selected
day selected
year selected
2019-07-31
page loading time = 0.14461302757263184 seconds
sleep
awake
{'date': datetime.date(2019, 7, 31), 'high_temp': '93', 'low_temp': '57', 'average_temp': '75'}
2019-08-01
month selected
day selected
year selected
2019-08-01
page loading time = 0.03789830207824707 seconds
sleep
awake
{'date': datetime.date(2019, 8, 1), 'high_temp': '90', 'low_temp': '59', 'average_temp': '74'}
2019-08-02
month selected
day selected
year selected
2019-08-02
page loading time = 0.03690004348754883 seconds
sleep
awake
{'date': datetime.date(

In [30]:
browser.quit()

In [31]:
f_handle.close()